# QM8 Dataset
[reference examples](https://github.com/deepchem/deepchem/tree/master/examples/qm8)

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import numpy as np
import tensorflow as tf
import sklearn 
import deepchem as dc

**Multitask Network**  

In [ ]:
qm8_tasks, qm8_datasets, transformers = dc.molnet.load_qm8_from_mat(
    featurizer='CoulombMatrix', split='random') # default
train_dataset, valid_dataset, test_dataset = qm8_datasets

In [ ]:
from dc.models import MultiTaskRegressor

def mtr_model_builder(model_params, model_dir):
    mtr_model = MultiTaskRegressor(**model_params)
    return mn_model

In [ ]:
metric = [
    dc.metrics.Metric(dc.metric.mean_absolute_error, mode="regression"),
    dc.metrics.Metric(dc.metric.pearson_r2_score, mode="regression")
]

params = { # TODO
}

model = mtr_model_builder(params, model_dir=None)

print("training MTR model")
model.fit(train_dataset, nb_epochs=50)

print("evaluating MTR model on {train,valid,test} sets")
train_scores = model.evaluate(train_dataset, metric, transformers)
print("train scores:\n" + str(train_scores))
valid_scores = model.evaluate(valid_dataset, metric, transformers)
print("valid scores:\n" + str(valid_scores))
test_scores = model.evaluate(test_dataset, metric, transformers)
print("test scores:\n" + str(test_scores))

**Kernel Ridge Regression**  

In [ ]:
qm8_tasks, qm8_datasets, transformers = dc.molnet.load_qm8_from_mat(
    featurizer='CoulombMatrix', split='random') # default
train_dataset, valid_dataset, test_dataset = qm8_datasets

In [ ]:
def krr_model_builder(model_params, model_dir):
    pass # TODO

In [ ]:
metric = [
] # TODO

params = {
} # TODO

model = krr_model_builder(params, model_dir=None)

print("training KRR model")
model.fit(train_dataset, nb_epochs=50)

print("evaluating KRR model on {train,valid,test} sets")
train_scores = model.evaluate(train_dataset, metric, transformers)
print("train scores:\n" + str(train_scores))
valid_scores = model.evaluate(valid_dataset, metric, transformers)
print("valid scores:\n" + str(valid_scores))
test_scores = model.evaluate(test_dataset, metric, transformers)
print("test scores:\n" + str(test_scores))

**Graph Convolution**  

In [ ]:
qm8_tasks, qm8_datasets, transformers = dc.molnet.load_qm8_from_mat(
    featurizer='CoulombMatrix', split='random') # default
train_dataset, valid_dataset, test_dataset = qm8_datasets

In [ ]:
def gc_model_builder(model_params, model_dir):
    pass

In [ ]:
metric = [
] # TODO

params = {
} # TODO

model = gc_model_builder(params, model_dir=None)

print("training GC model")
model.fit(train_dataset, nb_epochs=50)

print("evaluating GC model on {train,valid,test} sets")
train_scores = model.evaluate(train_dataset, metric, transformers)
print("train scores:\n" + str(train_scores))
valid_scores = model.evaluate(valid_dataset, metric, transformers)
print("valid scores:\n" + str(valid_scores))
test_scores = model.evaluate(test_dataset, metric, transformers)
print("test scores:\n" + str(test_scores))

**Multitask Network** _(Coulomb Matrix Featurizer)_  
[reference example](https://github.com/deepchem/deepchem/blob/master/examples/qm8/qm8_tf_model.py)

In [ ]:
qm8_tasks, qm8_datasets, transformers = dc.molnet.load_qm8_from_mat(
    featurizer='CoulombMatrix', split='random')
train_dataset, valid_dataset, test_dataset = qm8_datasets

In [ ]:
from dc.models import MultiTaskFitTransformRegressor

def mtftr_model_builder(model_params, model_dir):
    mtftr_model = MultiTaskFitTransformRegressor(**model_params)
    return mn_model

In [ ]:
fit_transformers = [
    dc.trans.CoulombFitTransformer(train_dataset)
]

metric = [
    dc.metrics.Metric(dc.metric.mean_absolute_error, mode="regression"),
    dc.metrics.Metric(dc.metric.pearson_r2_score, mode="regression")
]

params = {
    "n_tasks": len(qm8_tasks),
    "n_features": [26, 26],
    "weight_init_stddevs": [1.0/np.sqrt(400), 1.0/np.sqrt(100), 1.0/np.sqrt(100)],
    "bias_init_consts": [0.0, 0.0, 0.0]
    "layer_sizes": [400, 100, 100],
    "dropouts": [0.01, 0.01, 0.01],
    "fit_transformers": fit_transformers,
    "batch_size": 32,
    "learning_rate": 0.001,
    "momentum": 0.8,
    "n_evals": 10
}

model = mtftr_model_builder(params, model_dir=None)

print("training MTFTR model")
model.fit(train_dataset, nb_epochs=50)

print("evaluating MTFTR model on {train,valid,test} sets")
train_scores = model.evaluate(train_dataset, metric, transformers)
print("train scores: [kcal/mol]\n" + str(train_scores))
valid_scores = model.evaluate(valid_dataset, metric, transformers)
print("valid scores: [kcal / mold]\n" + str(valid_scores))
test_scores = model.evaluate(test_dataset, metric, transformers)
print("test scores: [kcal/mol]\n" + str(test_scores))


**Kernel Ridge Regression** _(Coulomb Matrix Featurizer)_  

In [ ]:
qm8_tasks, qm8_datasets, transformers = dc.molnet.load_qm8_from_mat(
    featurizer='Raw', split='stratified')
train_dataset, valid_dataset, test_dataset = qm8_datasets

In [ ]:
def krr_model_builder(model_params, model_dir):
    pass # TODO

In [ ]:
metric = [
] # TODO

params = {
} # TODO

model = krr_model_builder(params, model_dir=None)

print("training <> model")
model.fit(train_dataset, nb_epochs=50)

print("evaluating <> model on {train,valid,test} sets")
train_scores = model.evaluate(train_dataset, metric, transformers)
print("train scores:\n" + str(train_scores))
valid_scores = model.evaluate(valid_dataset, metric, transformers)
print("valid scores:\n" + str(valid_scores))
test_scores = model.evaluate(test_dataset, metric, transformers)
print("test scores:\n" + str(test_scores))

**Deep Tensor Neural Network**  
[reference example](https://github.com/deepchem/deepchem/blob/master/examples/qm8/qm8_DTNN.py)

In [ ]:
qm8_tasks, qm8_datasets, transformers = dc.molnet.load_qm8_from_mat(
    featurizer='Raw', split='stratified')
train_dataset, valid_dataset, test_dataset = qm8_datasets

In [ ]:
def dtnn_model_builder(model_params, model_dir):
    return dc.models.DTNNModel(output_activation=False,
                               use_queue=False,
                               mode="regression", 
                               **model_params)

In [ ]:
metric = [
    dc.metrics.Metric(dc.metric.pearson_r2_score, mode="regression")
]

params = {
    "n_tasks": len(qm8_tasks),
    "n_embedding": 20,
    "n_hidden": 15,
    "n_distance": 51,
    "distance_min": -1.0,
    "distance_max": 9.2,
    "batch_size": 50,
    "learning_rate": 0.0001
}

model = dtnn_model_builder(params, model_dir=None)

print("training DTNN model")
model.fit(train_dataset, nb_epoch=50)

print("evaluating DTNN model on {train,valid,test} sets")
train_scores = model.evaluate(train_dataset, metric, transformers)
print("train scores:\n" + str(train_scores))
valid_scores = model.evaluate(valid_dataset, metric, transformers)
print("valid scores:\n" + str(valid_scores))
test_scores = model.evaluate(test_dataset, metric, transformers)
print("test scores:\n" + str(test_scores))

**Message Passing Neural Network**  
[reference example](https://github.com/deepchem/deepchem/blob/master/examples/qm8/qm8_MPNN.py)

In [ ]:
qm8_tasks, qm8_datasets, transformers = dc.molnet.load_qm8_from_mat(
    featurizer='MP', split='random')
train_dataset, valid_dataset, test_dataset = qm8_datasets

In [ ]:
from dc.models import MPNNModel

def mpnn_model_builder(model_params, model_dir):
    return MPNNModel(use_queue=False,
                     mode="regression",
                     **model_params)

In [ ]:
metric = [
    dc.metrics.Metric(dc.metrics.pearson_r2_score, mode="regression")
]

params = {
    "n_tasks": len(qm9_tasks),
    "n_atom_feat": 70,
    "n_pair_feat": 8,
    "T": 5,
    "M": 10,
    "batch_size": 32,
    "learning_reate": 0.0001
}

model = mpnn_model_builder(params, model_dir=None)

print("training MPNN model")
model.fit(train_dataset, nb_epochs=100)

print("evaluating MPNN model on {train,valid,test} sets")
train_scores = model.evaluate(train_dataset, metric, transformers)
print("train scores:\n" + str(train_scores))
valid_scores = model.evaluate(valid_dataset, metric, transformers)
print("valid scores:\n" + str(valid_scores))
test_scores = model.evaluate(test_dataset, metric, transformers)
print("test scores:\n" + str(test_scores))